# 1. Getting Data From API

In [24]:
%load_ext autoreload
%autoreload 2
import requests
import pandas as pd
from config import settings
import data
import sqlite3
import matplotlib.pyplot as plt
from model import GarchModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = data.SQLRepository(connection = connection)
gm_FPT = GarchModel(ticker = 'FPT.VN', repo = repo,use_new_data = False)
gm_FPT.wrangle_data()
gm_FPT.fit(p =1, q = 1)
gm_FPT.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -523.330
Distribution:                  Normal   AIC:                           1054.66
Method:            Maximum Likelihood   BIC:                           1068.70
                                        No. Observations:                  247
Date:                Tue, May 28 2024   Df Residuals:                      246
Time:                        21:08:29   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.2090      0.144      1.455      0.146 [-7.260e-02,  0.491]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0363  2.487e-02      1.461      0.144 [-1.242e-02,8.508e-02]
alpha[1]       0.0000  1.406e-02      0.000      1.000 [-2.756e-02,2.756e-02]
beta[1]        0.9840  6.329e-03    155.467      0.000      [  0.972,  0.996]
=============================================================================

Covariance estimator: robust
"""

In [26]:
gm_FPT.data.tail()

date
2024-05-20   -1.115242
2024-05-21    2.255639
2024-05-22    1.470588
2024-05-23   -0.362319
2024-05-24   -4.072727
Name: returns, dtype: float64

In [27]:
pred = gm_FPT.predict_volatility(horizon = 5)
pred

{'2024-05-27T00:00:00': 1.5442791495679262,
 '2024-05-28T00:00:00': 1.5436811012744893,
 '2024-05-29T00:00:00': 1.5430924003527244,
 '2024-05-30T00:00:00': 1.5425129041036933,
 '2024-05-31T00:00:00': 1.541942471903951}

In [44]:
gm_FPT.load()

Go to the command line, navigate to the directory for this project, and start your app server by entering the following command.

```bash
uvicorn my_api:app --reload --workers 1 --host localhost --port 8008
```

In [46]:
url = "http://localhost:8008/hello"
response = requests.get(url = url)

print("response code:", response.status_code)
response.json()

response code: 200


{'message': 'Hello world!'}

In [49]:
# URL of '/fit' path
url = "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker": "FPT.VN",
    "use_new_data": False,
    "n_observations" : 200,
    "p": 1,
    "q" : 1
}

# Response of post request
response = requests.post(url = url, json = json)

# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'FPT.VNNNNN',
 'use_new_data': True,
 'n_observations': 200,
 'p': 1,
 'q': 1,
 'success': False,
 'message': 'Expecting value: line 1 column 1 (char 0)'}

In [53]:
# URL of `/predict` path
url = "http://localhost:8008/predict"
# Data to send to path
json = {'ticker':'FPT.VN', 'n_days': 5}
# Response of post request
response = requests.post(url = url, json = json)
# Response JSON to be submitted to grader
prediction = response.json()
# Inspect JSON
prediction

{'ticker': 'FPT.VN',
 'n_days': 5,
 'success': True,
 'forecast': {'2024-03-15T00:00:00': 1.2582860041070854,
  '2024-03-18T00:00:00': 1.2547396314184365,
  '2024-03-19T00:00:00': 1.2512236304315763,
  '2024-03-20T00:00:00': 1.2477377978019735,
  '2024-03-21T00:00:00': 1.2442819309389883},
 'message': ''}